In [1]:
import pandas as pd
from rdflib import Graph, URIRef

from viscars.data import DataLoader
from viscars.evaluation.evaluators.kfoldcv import KFoldCrossValidation
from viscars.evaluation.evaluators.loocv import LeaveOneOutCrossValidation
from viscars.evaluation.metrics import MetricType
from viscars.evaluation.metrics.factory import MetricFactory
from viscars.namespace import DASHB
from viscars.recommenders.factory import RecommenderFactory, RecommenderType
from viscars.utils import visualize_graph

In [3]:
project = 'proeftuin'

graph = Graph()
graph.parse(f'./data/{project}/graph.ttl', format='n3')
d_loader = DataLoader(graph)

metric_factory = MetricFactory()
recommender_factory = RecommenderFactory()

metrics = ['f1@1', 'ndcg@1', 'ndcg@3']
parsed_metrics = []
for metric in metrics:
    m_split = metric.split('@')
    m_type = m_split[0]
    n = int(m_split[1]) if len(m_split) >= 2 else None

    metric_ = metric_factory.get(MetricType.reverse_lookup(m_type), n)
    parsed_metrics.append(metric_)

C:\Users\Pieter\anaconda3\envs\VisCARS\lib\site-packages\viscars\data\__init__.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings = ratings.append({'user': user_, 'item': row[2], 'rating': 5.0, 'context': properties},
C:\Users\Pieter\anaconda3\envs\VisCARS\lib\site-packages\viscars\data\__init__.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings = ratings.append({'user': user_, 'item': row[2], 'rating': 5.0, 'context': properties},
C:\Users\Pieter\anaconda3\envs\VisCARS\lib\site-packages\viscars\data\__init__.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ratings = ratings.append({'user': user_, 'item': row[2], 'rating': 5.0, 'context': properties},
C:\Users\Pieter\anaconda3\envs\VisCARS\lib\site-p

In [15]:
recommender = recommender_factory.get(RecommenderType.PPR)(graph, alpha=0.3)
recommender.set_personalization(0.7, 0.3)

print(d_loader.user_metadata)
print(d_loader.item_metadata)

                                                id  \
0   https://dynamicdashboard.ilabt.imec.be/users/7   
1   https://dynamicdashboard.ilabt.imec.be/users/8   
2   https://dynamicdashboard.ilabt.imec.be/users/9   
3   https://dynamicdashboard.ilabt.imec.be/users/6   
4  https://dynamicdashboard.ilabt.imec.be/users/12   
5  https://dynamicdashboard.ilabt.imec.be/users/10   
6  https://dynamicdashboard.ilabt.imec.be/users/11   

                                                type  
0  https://dynamicdashboard.ilabt.imec.be/groups/...  
1  https://dynamicdashboard.ilabt.imec.be/groups/...  
2  https://dynamicdashboard.ilabt.imec.be/groups/...  
3  https://dynamicdashboard.ilabt.imec.be/groups/...  
4  https://dynamicdashboard.ilabt.imec.be/groups/...  
5  https://dynamicdashboard.ilabt.imec.be/groups/...  
6  https://dynamicdashboard.ilabt.imec.be/groups/...  
Empty DataFrame
Columns: [id]
Index: []


In [16]:
uid = ['https://dynamicdashboard.ilabt.imec.be/users/11']
cid = ['https://webthing.proeftuin.dynamicdashboard.ilabt.imec.be/things/BBB4/properties/rpm::number']

recommender.predict(uid, cid)

[]